# Data Cleaning and Integration

## Overview

This notebook demonstrates how I process, clean, and integrate daily COVID-19 datasets covering tests, cases, hospitalizations, deaths, and vaccinations (2020–2024).  


**Challenge**: Each data source came from a different person/system with different column names, date formats, missing values and inconsistent categories

**Solution**: Standardize data formats, handle missing data, validate data quality and create a unified dataset for analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os

os.chdir('..') #back to covid-analytics-portfolio project root
os.makedirs('../covid-analytics-portfolio/data/processed', exist_ok=True)
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Generate Raw Data

In the real project, I pulled data from 5 different databases daily. 
Below, I've created a sample data for demonstration.

In [2]:
dates = pd.date_range(start='2020-03-01', end='2024-01-31', freq='D')
n = len(dates)
    
df_raw = pd.DataFrame({
    'Date': dates,
    'Cases': np.random.poisson(100, n) + np.random.randint(-20, 20, n),
    'Tests': np.random.poisson(1000, n) + np.random.randint(-100, 100, n),
    'Hospitalizations': np.random.poisson(50, n) + np.random.randint(-10, 10, n),
    'Deaths': np.random.poisson(5, n),
    'Vaccinations': np.random.poisson(500, n)
})
    
df_raw.loc[np.random.choice(df_raw.index, 50), 'Cases'] = np.nan
df_raw.loc[np.random.choice(df_raw.index, 30), 'Hospitalizations'] = np.nan

os.makedirs('../covid-analytics-portfolio/data/raw', exist_ok=True)
df_raw.to_csv('../covid-analytics-portfolio/data/raw/covid_raw_data.csv', index=False)
    
print(f"Created sample data with {len(df_raw)} rows")
display(df_raw.head())

Created sample data with 1432 rows


,Date,Cases,Tests,Hospitalizations,Deaths,Vaccinations
0,2020-03-01,87.0,1131,53.0,5,516
1,2020-03-02,113.0,886,54.0,7,537
2,2020-03-03,112.0,1094,57.0,3,516
3,2020-03-04,111.0,962,53.0,8,547
4,2020-03-05,114.0,1072,23.0,2,506


## 2. Data Quality Assessment

Before cleaning, I always run basic descriptive checks.  

Generally, things I try to look for at this step:
- Missing values
- Data types
- Outliers
- Inconsistent formats
- Duplicates

Normally, in these datasets I often find:  
- Occasional missing days early in the pandemic  
- Slight reporting inconsistencies after holidays  
- Unexpected outliers (surges during local outbreaks)


This step helps me decide how to handle validations in later steps.

In [3]:
print(f"\nDataset Shape: {df_raw.shape[0]} rows x {df_raw.shape[1]} columns")
print(f"\nDate Range: {df_raw['Date'].min()} to {df_raw['Date'].max()}")

print("\nData Types:")
print(df_raw.dtypes)

print("\nMissing Values:")
missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

duplicates = df_raw.duplicated().sum()
print(f"\nDuplicate Rows: {duplicates}")

print("\nBasic Statistics:")
display(df_raw.describe())


Dataset Shape: 1432 rows x 6 columns

Date Range: 2020-03-01 00:00:00 to 2024-01-31 00:00:00

Data Types:
Date                datetime64[ns]
Cases                      float64
Tests                        int64
Hospitalizations           float64
Deaths                       int64
Vaccinations                 int64
dtype: object

Missing Values:
                  Missing Count  Percentage
Cases                        49        3.42
Hospitalizations             30        2.09

Duplicate Rows: 0

Basic Statistics:


,Cases,Tests,Hospitalizations,Deaths,Vaccinations
count,1383.000000,1432.000000,1402.000000,1432.000000,1432.00000
mean,99.132321,1000.724860,49.594151,5.022346,499.38338
std,15.053758,65.335149,9.338586,2.194473,22.99276
min,56.000000,824.000000,19.000000,0.000000,422.00000
25%,88.000000,951.000000,43.000000,3.000000,483.00000
50%,99.000000,1000.500000,49.000000,5.000000,500.00000
75%,110.000000,1052.000000,56.000000,6.000000,516.00000
max,140.000000,1171.000000,80.000000,14.000000,592.00000


## 3. Data Cleaning

For missing counts, I filled them with zeros rather than mean, since zeros most likely represent non-reporting days rather than true absences.  

1. Standardize column names
2. Convert date column to datetime
3. Remove duplicates
4. Handle missing value
5. Ensure correct data types
6. Sort by date


In [4]:
df_clean = df_raw.copy()

# 1. Standardize column names
df_clean.columns = df_clean.columns.str.lower().str.strip().str.replace(' ', '_')
print(f"Columns: {list(df_clean.columns)}")

# 2. Fix date format: convert date column to datetime
df_clean['date'] = pd.to_datetime(df_clean['date'], errors='coerce')
print(f"Date range: {df_clean['date'].min()} to {df_clean['date'].max()}")

# 3. Remove duplicates
before_dup = len(df_clean)
df_clean = df_clean.drop_duplicates()
removed_dup = before_dup - len(df_clean)
print(f"Removed {removed_dup} duplicate rows")

# 4. Handle missing values
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    missing_count = df_clean[col].isna().sum()
    if missing_count > 0:
        df_clean[col] = df_clean[col].fillna(0)
        print(f"Filled {missing_count} missing values in '{col}' with 0")

# 5. Ensure correct data types : convert numeric columns
for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0)

# 6. Sort by date
df_clean = df_clean.sort_values('date').reset_index(drop=True)

print("\nCleaning complete!")
print(f"\nFinal dataset: {len(df_clean)} rows x {len(df_clean.columns)} columns")


Columns: ['date', 'cases', 'tests', 'hospitalizations', 'deaths', 'vaccinations']
Date range: 2020-03-01 00:00:00 to 2024-01-31 00:00:00
Removed 0 duplicate rows
Filled 49 missing values in 'cases' with 0
Filled 30 missing values in 'hospitalizations' with 0

Cleaning complete!

Final dataset: 1432 rows x 6 columns


## 4. Data Integration

In the actual project, this is where I merged 5 separate data, and merging across inconsistent sources was a huge challenge.  

I settled on left joins to ensure my master timeline always had a row for each day, which simplified aggregations and later trend analyses.

In [5]:
# Simulate having separate sources
testing_data = df_clean[['date', 'tests']].copy()
case_data = df_clean[['date', 'cases']].copy()
hosp_data = df_clean[['date', 'hospitalizations']].copy()
death_data = df_clean[['date', 'deaths']].copy()
vacc_data = df_clean[['date', 'vaccinations']].copy()

print("Source:")
print(f"  Testing data: {len(testing_data)} rows")
print(f"  Case data: {len(case_data)} rows")
print(f"  Hospitalization data: {len(hosp_data)} rows")
print(f"  Death data: {len(death_data)} rows")
print(f"  Vaccination data: {len(vacc_data)} rows")

# Merge data sources
df_integrated = case_data.copy()
# Merge testing, hospitalization, death, vaccination data
df_integrated = df_integrated.merge(testing_data, on='date', how='left')
df_integrated = df_integrated.merge(hosp_data, on='date', how='left')
df_integrated = df_integrated.merge(death_data, on='date', how='left')
df_integrated = df_integrated.merge(vacc_data, on='date', how='left')
print("\nIntegration complete!")
print(f"\nIntegrated dataset: {len(df_integrated)} rows x {len(df_integrated.columns)} columns")

print("\nSample of integrated data:")
display(df_integrated.head(10))


Source:
  Testing data: 1432 rows
  Case data: 1432 rows
  Hospitalization data: 1432 rows
  Death data: 1432 rows
  Vaccination data: 1432 rows

Integration complete!

Integrated dataset: 1432 rows x 6 columns

Sample of integrated data:


,date,cases,tests,hospitalizations,deaths,vaccinations
0,2020-03-01,87.0,1131,53.0,5,516
1,2020-03-02,113.0,886,54.0,7,537
2,2020-03-03,112.0,1094,57.0,3,516
3,2020-03-04,111.0,962,53.0,8,547
4,2020-03-05,114.0,1072,23.0,2,506
5,2020-03-06,0.0,984,37.0,4,505
6,2020-03-07,87.0,969,49.0,7,535
7,2020-03-08,109.0,1061,45.0,5,551
8,2020-03-09,111.0,949,43.0,3,496
9,2020-03-10,105.0,990,50.0,5,507


## 5. Calculate Metrics


1. 7-day moving averages (smooth out daily fluctuations)
2. Positivity rate (cases / tests)
3. Week-over-week changes
4. Cumulative totals

Moving averages smoothed out daily reporting noise, and will reveal true peaks and valleys far better than raw data.  


In [6]:
df_final = df_integrated.copy()

# 1. Calculate 7-day moving averages
numeric_cols = ['cases', 'tests', 'hospitalizations', 'deaths', 'vaccinations']

for col in numeric_cols:
    df_final[f'{col}_7day_avg'] = df_final[col].rolling(window=7, min_periods=1).mean()

# 2. Calculate positivity rate
df_final['positivity_rate'] = (df_final['cases'] / df_final['tests'] * 100).fillna(0)
df_final['positivity_rate_7day_avg'] = df_final['positivity_rate'].rolling(window=7, min_periods=1).mean()

# 3. Calculate week-over-week change
for col in numeric_cols:
    df_final[f'{col}_wow_change'] = df_final[col] - df_final[col].shift(7)
    df_final[f'{col}_wow_pct'] = (df_final[f'{col}_wow_change'] / df_final[col].shift(7) * 100).fillna(0)


# 4. Calculate cumulative totals
for col in ['cases', 'deaths', 'vaccinations']:
    df_final[f'{col}_cumulative'] = df_final[col].cumsum()

print("\nAll metrics calculated!")
print(f"\nFinal dataset: {len(df_final)} rows x {len(df_final.columns)} columns")

print("\nSample with derived metrics:")
display(df_final[['date', 'cases', 'cases_7day_avg', 'positivity_rate', 'cases_wow_change']].tail(10))



All metrics calculated!

Final dataset: 1432 rows x 26 columns

Sample with derived metrics:


,date,cases,cases_7day_avg,positivity_rate,cases_wow_change
1422,2024-01-22,86.0,94.142857,8.113208,-33.0
1423,2024-01-23,66.0,86.857143,6.606607,-51.0
1424,2024-01-24,113.0,88.714286,10.741445,13.0
1425,2024-01-25,89.0,88.142857,9.694989,-4.0
1426,2024-01-26,112.0,91.428571,10.536218,23.0
1427,2024-01-27,104.0,93.714286,9.952153,16.0
1428,2024-01-28,97.0,95.285714,9.159585,11.0
1429,2024-01-29,92.0,96.142857,10.233593,6.0
1430,2024-01-30,98.0,100.714286,9.683794,32.0
1431,2024-01-31,83.0,96.428571,8.905579,-30.0


## 6. Data Validation

1. No negative values (except for change metrics)
2. Positivity rate between 0-100%
3. No date gaps
4. Cumulative values does not decrease
5. Testing > cases

During my time on the project, there was once where the case number exceeds testing. This is logically impossible and indicates data reporting issues or typos in the source. I later found out it was because the case data was being updated more frequently.

Catching these inconsistencies early helps prevent misleading downstream analysis and reporting.


In [7]:
validation_passed = True

# Make sure there's no negative values
print("Any negative values?")
count_cols = ['cases', 'tests', 'hospitalizations', 'deaths', 'vaccinations']
for col in count_cols:
    negative_count = (df_final[col] < 0).sum()
    if negative_count > 0:
        print(f"  WARNING: {col}: {negative_count} negative values found")
        validation_passed = False
    else:
        print(f"  OK: {col} no negative values")

# Positivity rate in valid range
print("\nPositivity rate in range?")
invalid_pos = ((df_final['positivity_rate'] < 0) | (df_final['positivity_rate'] > 100)).sum()
if invalid_pos > 0:
    print(f"  WARNING: {invalid_pos} invalid positivity rates found")
    validation_passed = False
else:
    print(f"  OK: All positivity rates in valid range (0-100%)")

# Make sure there are no gaps in dates
print("\nMissing dates?")
date_range = pd.date_range(start=df_final['date'].min(), end=df_final['date'].max(), freq='D')
missing_dates = set(date_range) - set(df_final['date'])
if len(missing_dates) > 0:
    print(f"  WARNING: {len(missing_dates)} missing dates found")
else:
    print(f"  OK: No date gaps found")


# Cumulative totals should not decrease
print("\nNo decrease in culmulative totals?")
cumulative_cols = ['cases_cumulative', 'deaths_cumulative', 'vaccinations_cumulative']
cumulative_issues = 0
for col in cumulative_cols:
    decreases = (df_final[col].diff() < 0).sum()
    if decreases > 0:
        print(f"  WARNING: {decreases} decreases detected in '{col}'")
        cumulative_issues += decreases
        validation_passed = False
    else:
        print(f"  OK: '{col}' does not decrease")

if cumulative_issues == 0:
    print("  All cumulative totals increase or remain flat as expected.")
    
    
# Check if cases exceed tests
print("\nTests > Cases?")
invalid_case_test = (df_final['cases'] > df_final['tests']).sum()
if invalid_case_test > 0:
    print(f"  WARNING: {invalid_case_test} rows where cases exceed tests found!")
    validation_passed = False
else:
    print("  OK: Cases never exceed tests")



if validation_passed:
    print("\nALL VALIDATION CHECKS PASSED")
else:
    print("\nSOME VALIDATION CHECKS FAILED - REVIEW NEEDED")

Any negative values?
  OK: cases no negative values
  OK: tests no negative values
  OK: hospitalizations no negative values
  OK: deaths no negative values
  OK: vaccinations no negative values

Positivity rate in range?
  OK: All positivity rates in valid range (0-100%)

Missing dates?
  OK: No date gaps found

No decrease in culmulative totals?
  OK: 'cases_cumulative' does not decrease
  OK: 'deaths_cumulative' does not decrease
  OK: 'vaccinations_cumulative' does not decrease
  All cumulative totals increase or remain flat as expected.

Tests > Cases?
  OK: Cases never exceed tests

ALL VALIDATION CHECKS PASSED


## Step 7: Save Cleaned Data


In [8]:
output_file = '../covid-analytics-portfolio/data/processed/covid_integrated_data.csv'
df_final.to_csv(output_file, index=False)

print(f"Cleaned data saved to: {output_file}")
print(f"\nDataset Summary:")
print(f"  Rows: {len(df_final):,}")
print(f"  Columns: {len(df_final.columns)}")
print(f"  Date range: {df_final['date'].min()} to {df_final['date'].max()}")
print(f"  File size: {os.path.getsize(output_file) / 1024:.2f} KB")

Cleaned data saved to: ../covid-analytics-portfolio/data/processed/covid_integrated_data.csv

Dataset Summary:
  Rows: 1,432
  Columns: 26
  Date range: 2020-03-01 00:00:00 to 2024-01-31 00:00:00
  File size: 386.53 KB


## Summary

### What was done in this notebook:
- Loaded data from multiple sources
- Assessed data quality issues
- Cleaned and standardized all data
- Integrated 5 different data sources
- Calculated derived metrics and moving averages
- Validated data quality
- Saved cleaned data for analysis

### Key learnings:
- Always inspect raw data before jumping to cleaning.
- Simple imputation strategies go a long way — but validate against business logic.
- Documenting even ‘obvious’ pipeline stages helps future you (and others) debug faster!
